In [1]:
import pandas as pd
import numpy as np

#countvectorizer model
import nltk
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer


from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO

from langdetect import detect

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv("../data/goodreads_final.csv")
df.head()

,url,bookTitle,bookImage,bookDesc,bookRating,ratingCount,reviewCount,Genre,pageCount,Author
0,https://www.goodreads.com/book/show/2767052-the-hunger-games,The Hunger Games,https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1586722975l/2767052.jpg,"Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.",4.32,6717635,176054,Young Adult,374,Suzanne Collins
1,https://www.goodreads.com/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1546910265l/2.jpg,"There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.His fate depends on them all.",4.50,2668409,45724,Fantasy,870,J.K. Rowling
2,https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1553383690l/2657.jpg,"The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. ""To Kill A Mockingbird"" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, ""To Kill A Mockingbird"" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.",4.28,4772918,95595,Classics,324,Harper Lee
3,https://www.goodreads.com/book/show/1885.Pride_and_Prejudice,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1320399351l/1885.jpg,"Alternate cover edition of ISBN 9780679783268Since its immediate success in 1813, Pride and Prejudice has remained one of the most popular novels in the English language. Jane Austen called this brilliant work ""her own darling child"" and its vivacious heroine, Elizabeth Bennet, ""as delightful a creature as ever appeared in print."" The romantic clash between the opinionated Elizabeth and her proud beau, Mr. Darcy, is a splendid performance of civilized sparring. And Jane Auste

In [3]:
df["lang"] = df.bookDesc.apply(lambda x: detect(x))
df.head()

,url,bookTitle,bookImage,bookDesc,bookRating,ratingCount,reviewCount,Genre,pageCount,Author,lang
0,https://www.goodreads.com/book/show/2767052-the-hunger-games,The Hunger Games,https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1586722975l/2767052.jpg,"Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.",4.32,6717635,176054,Young Adult,374,Suzanne Collins,en
1,https://www.goodreads.com/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1546910265l/2.jpg,"There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.His fate depends on them all.",4.50,2668409,45724,Fantasy,870,J.K. Rowling,en
2,https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1553383690l/2657.jpg,"The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. ""To Kill A Mockingbird"" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, ""To Kill A Mockingbird"" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.",4.28,4772918,95595,Classics,324,Harper Lee,en
3,https://www.goodreads.com/book/show/1885.Pride_and_Prejudice,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1320399351l/1885.jpg,"Alternate cover edition of ISBN 9780679783268Since its immediate success in 1813, Pride and Prejudice has remained one of the most popular novels in the English language. Jane Austen called this brilliant work ""her own darling child"" and its vivacious heroine, Elizabeth Bennet, ""as delightful a creature as ever appeared in print."" The romantic clash between the opinionated Elizabeth and her proud beau, Mr. Darcy, is a splendid performance of civilized sparring. 

In [4]:
df.lang.value_counts()

en    9350
ar      90
es      82
fr      54
de      36
id      35
pt      21
it      17
tr      15
fa      12
pl      11
nl       8
ro       6
bg       5
ja       5
hr       5
no       3
el       3
ru       3
fi       2
cs       2
ta       1
sv       1
sk       1
uk       1
af       1
tl       1
et       1
Name: lang, dtype: int64

In [5]:
df_sub = df.loc[df.lang == "en"]
df_sub.bookTitle.nunique()

8863

In [6]:
9350- 8863

487

## EDA

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9772 entries, 0 to 9771
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          9772 non-null   object 
 1   bookTitle    9772 non-null   object 
 2   bookImage    9772 non-null   object 
 3   bookDesc     9772 non-null   object 
 4   bookRating   9772 non-null   float64
 5   ratingCount  9772 non-null   int64  
 6   reviewCount  9772 non-null   int64  
 7   Genre        9772 non-null   object 
 8   pageCount    9772 non-null   int64  
 9   Author       9772 non-null   object 
 10  lang         9772 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 839.9+ KB


In [8]:
df["bookTitle"].nunique()

9272

In [9]:
df["Author"].nunique()

4140

In [10]:
df["Genre"].nunique()

46

In [11]:
df["Genre"].value_counts()

Fiction                          1861
Fantasy                          1857
Young Adult                      1042
Romance                           650
Classics                          559
Historical-Historical Fiction     504
Nonfiction                        425
Science Fiction                   372
Mystery                           320
Horror                            227
Religion                          178
Sequential Art-Graphic Novels     176
Fantasy-Paranormal                118
Paranormal-Vampires               106
Poetry                            101
History                            98
Thriller                           97
Philosophy                         92
Other                              89
Romance-Paranormal Romance         88
Womens Fiction-Chick Lit           69
Childrens                          65
Short Stories                      56
New Adult                          46
Science                            45
Contemporary                       45
Plays       

In [ ]:
#review some for similarity
dfNew.loc[dfNew.bookTitle == 'Fahrenheit 451']

In [ ]:
dfNew.loc[dfNew.bookTitle == "Apollyon"]
#different titles in data set

In [ ]:
dfNew.loc[dfNew.Author == "J.K. Rowling"]

### dataframe for analysis

In [13]:
#copy book title, book description, genre, and author
dfNew = df[["bookTitle", "bookDesc", "Genre", "Author"]].copy()
dfNew.head()

,bookTitle,bookDesc,Genre,Author
0,The Hunger Games,"Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.",Young Adult,Suzanne Collins
1,Harry Potter and the Order of the Phoenix,"There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.His fate depends on them all.",Fantasy,J.K. Rowling
2,To Kill a Mockingbird,"The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. ""To Kill A Mockingbird"" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, ""To Kill A Mockingbird"" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.",Classics,Harper Lee
3,Pride and Prejudice,"Alternate cover edition of ISBN 9780679783268Since its immediate success in 1813, Pride and Prejudice has remained one of the most popular novels in the English language. Jane Austen called this brilliant work ""her own darling child"" and its vivacious heroine, Elizabeth Bennet, ""as delightful a creature as ever appeared in print."" The romantic clash between the opinionated Elizabeth and her proud beau, Mr. Darcy, is a splendid performance of civilized sparring. And Jane Austen's radiant wit sparkles as her characters dance a delicate quadrille of flirtation and intrigue, making this book the most superb comedy of manners of Regency England.",Classics,Jane Austen
4,Twilight,"About three things I was absolutely positive.First, Edward was a vampire.Second, there was a part of him—and I didn't know how dominant that part might be—that thirsted for my blood.And third, I was unconditionally and irrevocably in love with him.Deeply seductive and extraordinarily suspenseful, Twilight is a love story with bite.",Young Adult,Stephenie Meyer


In [14]:
dfNew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9772 entries, 0 to 9771
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   bookTitle  9772 non-null   object
 1   bookDesc   9772 non-null   object
 2   Genre      9772 non-null   object
 3   Author     9772 non-null   object
dtypes: object(4)
memory usage: 305.5+ KB


## Data Processing and Bag of Words

In [16]:
#data pre-processing to remove stop words, punctuation, white space, and convert all words to lower case from book Description
df_new = dfNew.copy()
df_new['Key_words'] = ''
df_new['bookTitle_key'] = ''
r = Rake()

for index, row in df_new.iterrows():
    r.extract_keywords_from_text(row['bookDesc'])
    key_words_dict_scores = r.get_word_degrees()
    row['Key_words'] = list(key_words_dict_scores.keys())

    r.extract_keywords_from_text(row['bookTitle'])
    key_words_dict_scores = r.get_word_degrees()
    row['bookTitle_key'] = list(key_words_dict_scores.keys())
    
    
df_new.head()

,bookTitle,bookDesc,Genre,Author,Key_words,bookTitle_key
0,The Hunger Games,"Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.",Young Adult,Suzanne Collins,"[could, survive, wild, every, one, make, sure, live, see, morning, ruins, place, known, north, america, lies, nation, panem, shining, capitol, surrounded, twelve, outlying, districts, harsh, cruel, keeps, line, forcing, send, boy, girl, ages, eighteen, participate, annual, hunger, games, fight, death, tv, sixteen, year, old, katniss, everdeen, lives, alone, mother, younger, sister, regards, sentence, steps, forward, take, close, dead, —, survival, second, nature, without, really, meaning, becomes, contender, win, start, making, choices, weight, humanity, life, love]","[hunger, games]"
1,Harry Potter and the Order of the Phoenix,"There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.His fate depends on them all.",Fantasy,J.K. Rowling,"[door, end, silent, corridor, ’, haunting, harry, pottter, dreams, else, would, waking, middle, night, screaming, terror, lot, mind, fifth, year, hogwarts, defense, dark, arts, teacher, personality, like, poisoned, honey, big, surprise, gryffindor, quidditch, team, looming, ordinary, wizarding, level, exams, things, pale, next, growing, threat, must, named, neither, magical, government, authorities, stop, grasp, darkness, tightens, discover, true, depth, strength, friends, importance, boundless, loyalty, shocking, price, unbearable, sacrifice, fate, depends]","[harry, potter, order, phoenix]"
2,To Kill a Mockingbird,"The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. ""To Kill A Mockingbird"" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, ""To Kill A Mockingbird"" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.",Classics,Harper Lee,"[unforgettable, novel, childhood, sleepy, sout

In [17]:
#DROP SYMBOLS

In [18]:
df_new2 = df_new.copy()

df_new2.head()

,bookTitle,bookDesc,Genre,Author,Key_words,bookTitle_key
0,The Hunger Games,"Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.",Young Adult,Suzanne Collins,"[could, survive, wild, every, one, make, sure, live, see, morning, ruins, place, known, north, america, lies, nation, panem, shining, capitol, surrounded, twelve, outlying, districts, harsh, cruel, keeps, line, forcing, send, boy, girl, ages, eighteen, participate, annual, hunger, games, fight, death, tv, sixteen, year, old, katniss, everdeen, lives, alone, mother, younger, sister, regards, sentence, steps, forward, take, close, dead, —, survival, second, nature, without, really, meaning, becomes, contender, win, start, making, choices, weight, humanity, life, love]","[hunger, games]"
1,Harry Potter and the Order of the Phoenix,"There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.His fate depends on them all.",Fantasy,J.K. Rowling,"[door, end, silent, corridor, ’, haunting, harry, pottter, dreams, else, would, waking, middle, night, screaming, terror, lot, mind, fifth, year, hogwarts, defense, dark, arts, teacher, personality, like, poisoned, honey, big, surprise, gryffindor, quidditch, team, looming, ordinary, wizarding, level, exams, things, pale, next, growing, threat, must, named, neither, magical, government, authorities, stop, grasp, darkness, tightens, discover, true, depth, strength, friends, importance, boundless, loyalty, shocking, price, unbearable, sacrifice, fate, depends]","[harry, potter, order, phoenix]"
2,To Kill a Mockingbird,"The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. ""To Kill A Mockingbird"" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, ""To Kill A Mockingbird"" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.",Classics,Harper Lee,"[unforgettable, novel, childhood, sleepy, sout

In [19]:
#make columns lower case

df_new2['Genre'] = df_new2['Genre'].str.lower()
df_new2['Author'] = df_new2['Author'].str.lower()

#make columns one string
df_new2['Genre'] = df_new2['Genre'].str.replace(" ","")
df_new2['Author'] = df_new2['Author'].str.replace(" ","")

#add brackets for processing in next step
df_new2['Genre'] = df_new2['Genre'].apply(lambda x: x.split(','))
df_new2['Author'] = df_new2['Author'].apply(lambda x: x.split(','))

df_new2.head()

,bookTitle,bookDesc,Genre,Author,Key_words,bookTitle_key
0,The Hunger Games,"Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.",[youngadult],[suzannecollins],"[could, survive, wild, every, one, make, sure, live, see, morning, ruins, place, known, north, america, lies, nation, panem, shining, capitol, surrounded, twelve, outlying, districts, harsh, cruel, keeps, line, forcing, send, boy, girl, ages, eighteen, participate, annual, hunger, games, fight, death, tv, sixteen, year, old, katniss, everdeen, lives, alone, mother, younger, sister, regards, sentence, steps, forward, take, close, dead, —, survival, second, nature, without, really, meaning, becomes, contender, win, start, making, choices, weight, humanity, life, love]","[hunger, games]"
1,Harry Potter and the Order of the Phoenix,"There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.His fate depends on them all.",[fantasy],[j.k.rowling],"[door, end, silent, corridor, ’, haunting, harry, pottter, dreams, else, would, waking, middle, night, screaming, terror, lot, mind, fifth, year, hogwarts, defense, dark, arts, teacher, personality, like, poisoned, honey, big, surprise, gryffindor, quidditch, team, looming, ordinary, wizarding, level, exams, things, pale, next, growing, threat, must, named, neither, magical, government, authorities, stop, grasp, darkness, tightens, discover, true, depth, strength, friends, importance, boundless, loyalty, shocking, price, unbearable, sacrifice, fate, depends]","[harry, potter, order, phoenix]"
2,To Kill a Mockingbird,"The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. ""To Kill A Mockingbird"" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, ""To Kill A Mockingbird"" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.",[classics],[harperlee],"[unforgettable, novel, childhood, slee

In [20]:
#create a bag of words columns with processed columns
df_new3 = df_new2.copy()

df_new3['Bag_of_words'] = ''

columns = ['Genre', 'Author', 'bookTitle_key', 'Key_words']

for index, row in df_new3.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' '
    row['Bag_of_words'] = words
    
bag_words = df_new3[['bookTitle','Bag_of_words']]
bag_words.head()

,bookTitle,Bag_of_words
0,The Hunger Games,youngadult suzannecollins hunger games could survive wild every one make sure live see morning ruins place known north america lies nation panem shining capitol surrounded twelve outlying districts harsh cruel keeps line forcing send boy girl ages eighteen participate annual hunger games fight death tv sixteen year old katniss everdeen lives alone mother younger sister regards sentence steps forward take close dead — survival second nature without really meaning becomes contender win start making choices weight humanity life love
1,Harry Potter and the Order of the Phoenix,fantasy j.k.rowling harry potter order phoenix door end silent corridor ’ haunting harry pottter dreams else would waking middle night screaming terror lot mind fifth year hogwarts defense dark arts teacher personality like poisoned honey big surprise gryffindor quidditch team looming ordinary wizarding level exams things pale next growing threat must named neither magical government authorities stop grasp darkness tightens discover true depth strength friends importance boundless loyalty shocking price unbearable sacrifice fate depends
2,To Kill a Mockingbird,classics harperlee kill mockingbird unforgettable novel childhood sleepy southern town crisis conscience rocked kill mockingbird became instant bestseller critical success first published 1960 went win pulitzer prize 1961 later made academy award winning film also classic compassionate dramatic deeply moving takes readers roots human behavior innocence experience kindness cruelty love hatred humor pathos 18 million copies print translated forty languages regional story young alabama woman claims universal appeal harper lee always considered book simple today regarded masterpiece american literature
3,Pride and Prejudice,"classics janeausten pride prejudice alternate cover edition isbn 9780679783268since immediate success 1813 pride prejudice remained one popular novels english language jane austen called brilliant work darling child vivacious heroine elizabeth bennet delightful creature ever appeared print ."" romantic clash opinionated proud beau mr darcy splendid performance civilized sparring radiant wit sparkles characters dance delicate quadrille flirtation intrigue making book superb comedy manners regency england"
4,Twilight,youngadult stepheniemeyer twilight three things absolutely positive first edward vampire second part — know dominant might thirsted blood third unconditionally irrevocably love deeply seductive extraordinarily suspenseful twilight story bite


## Count Vectorizer

In [21]:
#CountVectorizer model
count = CountVectorizer()
count_matrix = count.fit_transform(bag_words['Bag_of_words'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)


[[1.         0.01275153 0.02424287 ... 0.0208696  0.03727355 0.07250185]
 [0.01275153 1.         0.         ... 0.04364358 0.02598279 0.03032392]
 [0.02424287 0.         1.         ... 0.0207435  0.03704833 0.04323826]
 ...
 [0.0208696  0.04364358 0.0207435  ... 1.         0.         0.02481458]
 [0.03727355 0.02598279 0.03704833 ... 0.         1.         0.08863877]
 [0.07250185 0.03032392 0.04323826 ... 0.02481458 0.08863877 1.        ]]


In [38]:
test = count_matrix.toarray()
test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [39]:
len(test[0])

74424

In [40]:
sum(test[0])

78

In [23]:
indices = pd.Series(bag_words['bookTitle'])

In [28]:
def recommend(bookTitle, cosine_sim = cosine_sim):
    recommended_book = []
    idx = indices[indices == bookTitle].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:21].index)
    
    for i in top_10_indices:
        recommended_book.append(list(bag_words['bookTitle'])[i])
        
    return recommended_book

In [29]:
recommend('To Kill a Mockingbird')

['To Kill a Mockingbird',
 'Go Set a Watchman',
 'Lonesome Dove',
 'The Chronicles of Prydain',
 'The Grapes of Wrath',
 'Our Town',
 'Lady of Hay',
 'The Secret Garden',
 'Manchild in the Promised Land',
 'Home',
 'Story of a Soul: The Autobiography of St. Thérèse of Lisieux',
 'Jimmy Corrigan: The Smartest Kid on Earth',
 'The Gift of the Magi',
 'The Perks of Being a Wallflower',
 'Jennie Gerhardt',
 'Lila',
 'The Garden of Eden',
 '...And Ladies of the Club',
 'Songs of Innocence and of Experience',
 'For Colored Girls Who Have Considered Suicide When the Rainbow Is Enuf']

In [ ]:
bagwords2 = bag_words.copy()
bagwords2 = bagwords2.drop_duplicates(subset=['bookTitle'])
bagwords2.info()

In [ ]:
bagwords2.loc[bagwords2['bookTitle']== "Apollyon"]

In [ ]:
#CountVectorizer model
count = CountVectorizer()
count_matrix = count.fit_transform(bagwords2['Bag_of_words'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)
indices = pd.Series(bagwords2['bookTitle'])

def recommend(bookTitle, cosine_sim = cosine_sim):
    recommended_book = []
    idx = indices[indices == bookTitle].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_book.append(list(bagwords2['bookTitle'])[i])
        
    return recommended_book


In [ ]:
recommend('Apollyon')

## TF- IDF

In [ ]:
bag_words.head()

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [32]:
#https://practicaldatascience.co.uk/data-science/how-to-create-content-recommendations-using-tf-idf
#bag of words is already made, but this will also down-weight common words that appear across documents
tfidf = TfidfVectorizer(stop_words='english')

# numbers to calculate similarities
tfidf_matrix = tfidf.fit_transform(bag_words['Bag_of_words'])

#calculate cosine matrix
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
#Return a dataframe of content recommendations based on TF-IDF cosine similarity.


def get_recommendations(title, cosine_similarities, limit=10):

    
    # Return indices for the target dataframe column and drop any duplicates
    indices = pd.Series(bag_words.index, index=bag_words['bookTitle']).drop_duplicates()

    # Get the index for the target value
    target_index = indices[title]

    # Get the cosine similarity scores for the target value
    cosine_similarity_scores = list(enumerate(cosine_similarities[target_index]))

    # Sort the cosine similarities in order of closest similarity
    cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)

    # Return tuple of the requested closest scores excluding the target item and index
    cosine_similarity_scores = cosine_similarity_scores[1:limit+1]

    # Extract the tuple values
    index = (x[0] for x in cosine_similarity_scores)
    scores = (x[1] for x in cosine_similarity_scores)    

    # Get the indices for the closest items
    recommendation_indices = [i[0] for i in cosine_similarity_scores]

    # Get the actutal recommendations
    recommendations = bag_words['bookTitle'].iloc[recommendation_indices]

    # Return a dataframe
    df = pd.DataFrame(list(zip(index, recommendations, scores)), 
                      columns=['index','recommendation', 'cosine_similarity_score']) 

    return df

In [ ]:
get_recommendations('The Hunger Games', cosine_similarities)

In [ ]:
# Return indices for the target dataframe column and drop any duplicates
indices = pd.Series(bag_words.index, index=bag_words['bookTitle']).drop_duplicates()

# Get the index for the target value
target_index = indices['The Hunger Games']
target_index

## Sentiment Analysis

In [ ]:
#genre as sentiment
sent_df = df_new2[["bookTitle","Genre", "Key_words"]].copy()
sent_df.head()

In [ ]:
df_new2["Genre"].value_counts()

## Sentiment Analysis w/ Support Vector Machines (SVM)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
from scikitplot.metrics import plot_confusion_matrix


In [ ]:
#object of WordNetLemmatizer
lm = WordNetLemmatizer()
def text_transformation(df_col):
    corpus = []
    for item in df_col:
        new_item = re.sub('[^a-zA-Z]',' ',str(item))
        new_item = new_item.lower()
        new_item = new_item.split()
        new_item = [lm.lemmatize(word) for word in new_item if word not in set(stopwords.words('english'))]
        corpus.append(' '.join(str(x) for x in new_item))
    return corpus
corpus = text_transformation(sent_df['Key_words'])


## Sentiment Analysis w/ Naive Bayes Algorithm

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

In [ ]:
count_matrix

In [ ]:
#apply TF_IDF Transformation to the bag of words columns (post CountVectorizer)
tfidf = TfidfTransformer() # by default applies "l2" normalization
X_tfidf = tfidf.fit_transform(count_matrix)
X_tfidf = X_tfidf.todense()
X_tfidf

In [ ]:

#split training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, 
                                                    test_size = 0.25, 
                                                    random_state = 0)